In [1]:
import sys
sys.path.append('../')
import wrangle
import explore
import nlp

In [2]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

df, data_dict = wrangle.wrangle_data()

In [4]:
df = explore.add_target_to_df(df)

targetB = df.targetb

targetB = targetB.reset_index().drop('resp_id',axis=1)

In [5]:
big_df = pd.read_csv('topics.csv', index_col=False)

big_df.drop('Unnamed: 0', axis=1,inplace=True)

In [6]:
big_df['targetB'] = targetB

In [7]:
likely = big_df[big_df.targetB == 1]

In [8]:
likely.shape

(372, 32)

In [9]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference', 'yes', 've', 'ha']

stopWords = nlp.set_stop_words(stop_words)

# Respondents Likely To Go

## Sentiment

In [ ]:
likely.big_answer.apply(nlp.find_polarity).mean()

In [ ]:
likely.big_answer.apply(nlp.find_subjectivity).mean()

## TOTAL KEYWORDS

In [ ]:
nlp.show_column_keywords(likely.big_answer, max_df=.8, stop_words=stopWords, ngram_range=(1,3))

## What does your company do?

q5 primary industry

### keywords

In [ ]:
likely.prim_ind_text

In [ ]:
#words
nlp.show_column_keywords(likely.prim_ind_text, max_df=.5, stop_words=stopWords, ngram_range=(1,3))

In [ ]:
#frequency
prim_ind_df = nlp.fi

In [ ]:
#ngrams
nlp.show_column_keywords(likely.prim_ind_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

### topics

In [ ]:
likely[likely.prim_ind_text.notnull()].prim_ind_topic_id.value_counts()

## What kind of research are you doing?

q7 future_res

### keywords

In [ ]:
#words
nlp.show_column_keywords(likely.future_res_text, max_df=.5, stop_words=stopWords, ngram_range=(1,3))

In [ ]:
#ngrams
nlp.show_column_keywords(likely.future_res_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

### topics

In [ ]:
likely[likely.future_res_text.notnull()].future_res_topic_id.value_counts()

## What kind of research are you doing?

q6 types_res_used

### keywords

In [ ]:
#words
nlp.show_column_keywords(likely.types_res_text, max_df=.5, stop_words=stopWords, ngram_range=(1,3))

In [ ]:
#ngrams
nlp.show_column_keywords(likely.types_res_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

### topics

In [ ]:
likely[likely.types_res_text.notnull()].types_res_topic_id.value_counts()

## What topics would they be most attracted to at a conference about research?

q21 ideal_topics

In [ ]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference', 'good', 'best', 'self', 'report']

In [ ]:
stopWords = nlp.set_stop_words(stop_words)

### keywords

In [ ]:
#words
nlp.show_column_keywords(likely.ideal_topics_text, max_df=.5, stop_words=stopWords, ngram_range=(1,3))

In [ ]:
#bi-grams
nlp.show_column_keywords(likely.ideal_topics_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

### topics

In [ ]:
likely[likely.ideal_topics_text.notnull()].ideal_topics_topic_id.value_counts()

## Who would they expect to see at a conference about research?

q22 Ideal Attendees

In [ ]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference']

stopWords = nlp.set_stop_words(stop_words)

In [ ]:
#terms
nlp.show_column_keywords(likely.ideal_attendees_text, max_df=.5, stop_words=stopWords)

In [ ]:
#bigrams
nlp.show_column_keywords(likely.ideal_attendees_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

In [ ]:
likely[likely.ideal_attendees_text.notnull()].ideal_attendees_topic_id.value_counts()

## What advice do they have for the Rosenfeld Media team in pursuing a conference?
q23 recommendations

In [ ]:
#words
nlp.show_column_keywords(likely.recommendations_text, max_df=.9, stop_words=stopWords, ngram_range=(1,7))

In [ ]:
#ngrams
nlp.show_column_keywords(likely.recommendations_text, max_df=.9, stop_words=stopWords, ngram_range=(2,7))

In [ ]:
likely[likely.recommendations_text.notnull()].recommendations_topic_id.value_counts()

## Top Documents per Topic

In [ ]:
doc_term_matrix, count_vect = nlp.create_wordcount_matrix(likely.recommendations_text, max_df=.3, ngram=(1,3), stop_words=stopWords)

LDA = LatentDirichletAllocation(n_components=4, random_state=42)

LDA.fit(doc_term_matrix)

In [ ]:
recommendations_dict = {0 : 'speaker, industry', 
                        1 : 'event, opportunity', 
                        2 : 'good, know', 
                        3 : 'field, survery', 
                        4 : 'group, career, city'}

In [ ]:
nlp.find_top_documents_per_topic(LDA.transform(doc_term_matrix), likely.recommendations_text, 5)